In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

Trabajamos en un colegio y tenemos las notas de Matemáticas, Inglés y Física de los alumnos del colegio en 3 documentos txt

1. Crea 3 RDD de pares, uno para cada asignatura, con los alumnos y sus notas

In [11]:
notasMat = sc.textFile('ejercicios/notas_mates.txt').map(lambda x:x.split(","))
notasFis = sc.textFile('ejercicios/notas_fisica.txt').map(lambda x:x.split(","))
notasIng = sc.textFile('ejercicios/notas_ingles.txt').map(lambda x:x.split(","))

2. Crea un solo RDD con todas las notas

In [12]:
notas = notasMat.union(notasFis).union(notasIng)

3. ¿Cuál es la nota más baja que ha tenido cada alumno?

In [44]:
notasMinAlumnos = notas.reduceByKey(min)
notasMinAlumnos.collect()

[('Angel', '4'),
 ('Carlos', '4'),
 ('Anabel', '2'),
 ('Jorge', '10'),
 ('Susana', '2'),
 ('Andres', '4'),
 ('Jose Juan', '3'),
 ('Rocio', '4'),
 ('Fernando', '5'),
 ('Oscar', '3'),
 ('Isabel', '7'),
 ('Alejandro', '3'),
 ('Ramon', '4.5'),
 ('Pedro', '2'),
 ('Leonardo', '1'),
 ('Nicolas', '2'),
 ('Maria', '2'),
 ('Triana', '3'),
 ('Rosa', '6')]

4. ¿Cuál es la nota media de cada alumno?

In [18]:
alumnoNota1 = notas.mapValues(lambda x: (float(x),1))
# Sumamos las notas [0] y contamos las notas [1]
alumnoNotaCant = alumnoNota1.reduceByKey(lambda alu1, alu2:(alu1[0]+alu2[0], alu1[1]+alu2[1]))
alumnoNotaCant.map(lambda x: (x[0],x[1][0]/x[1][1])).collect()

[('Angel', 6.333333333333333),
 ('Carlos', 5.333333333333333),
 ('Anabel', 5.666666666666667),
 ('Jorge', 6.666666666666667),
 ('Susana', 6.666666666666667),
 ('Andres', 4.666666666666667),
 ('Jose Juan', 3.6666666666666665),
 ('Rocio', 5.5),
 ('Fernando', 7.0),
 ('Oscar', 5.0),
 ('Isabel', 7.666666666666667),
 ('Alejandro', 5.0),
 ('Ramon', 6.5),
 ('Pedro', 3.5),
 ('Leonardo', 3.6666666666666665),
 ('Nicolas', 4.666666666666667),
 ('Maria', 3.6666666666666665),
 ('Triana', 3.3333333333333335),
 ('Rosa', 7.666666666666667)]

5. ¿Cuantos estudiantes suspende cada asignatura? (`[('Mates', 7), ('Física', 8), ('Inglés', 7)]`)

In [20]:
notasAsig = notasMat.map(lambda x: ("Mates", x)).union(notasFis.map(lambda x: ("Física", x))).union(notasIng.map(lambda x: ("Inglés", x)))
notasAsig.collect()

[('Mates', ['Angel', '6']),
 ('Mates', ['Maria', '2']),
 ('Mates', ['Ramon', '4.5']),
 ('Mates', ['Jorge', '10']),
 ('Mates', ['Susana', '9']),
 ('Mates', ['Anabel', '8']),
 ('Mates', ['Pedro', '5']),
 ('Mates', ['Rocio', '6']),
 ('Mates', ['Carlos', '4']),
 ('Mates', ['Triana', '3']),
 ('Mates', ['Andres', '4']),
 ('Mates', ['Fernando', '5']),
 ('Mates', ['Leonardo', '1']),
 ('Mates', ['Oscar', '7']),
 ('Mates', ['Isabel', '8']),
 ('Mates', ['Jose Juan', '5']),
 ('Mates', ['Nicolas', '2']),
 ('Mates', ['Alejandro', '5']),
 ('Mates', ['Rosa', '6']),
 ('Física', ['Angel', '9']),
 ('Física', ['Maria', '3']),
 ('Física', ['Ramon', '7']),
 ('Física', ['Jorge', '5']),
 ('Física', ['Susana', '9']),
 ('Física', ['Anabel', '2']),
 ('Física', ['Pedro', '2']),
 ('Física', ['Rocio', '5']),
 ('Física', ['Carlos', '4']),
 ('Física', ['Rocio', '7']),
 ('Física', ['Triana', '3']),
 ('Física', ['Andres', '4']),
 ('Física', ['Fernando', '9']),
 ('Física', ['Leonardo', '6']),
 ('Física', ['Oscar', '5'])

In [34]:
asigSusp = notasAsig.filter(lambda x: float(x[1][1])<5)
asigSuspCant = asigSusp.map(lambda x: (x[0], 1))
asigSusp = asigSuspCant.reduceByKey(lambda a,b: a+b)
asigSusp.collect()

[('Mates', 7), ('Física', 8), ('Inglés', 7)]

6. ¿En qué asignatura suspende más gente?

In [43]:
asigSusp.takeOrdered(1, lambda x: -x[1])

[('Física', 8)]

7. Total de notables o sobresalientes por alumno, es decir, cantidad de notas superiores o igual a 7.

In [48]:
notablesRDD = notas.filter(lambda x: float(x[1])>=7)
notablesRDD.map(lambda x: (x[0],1)).reduceByKey(lambda a,b: a+b).collect()

[('Angel', 1),
 ('Carlos', 1),
 ('Anabel', 2),
 ('Jorge', 1),
 ('Susana', 2),
 ('Oscar', 1),
 ('Isabel', 3),
 ('Rocio', 1),
 ('Fernando', 2),
 ('Alejandro', 1),
 ('Ramon', 2),
 ('Nicolas', 1),
 ('Rosa', 2)]

8. ¿Qué alumno no se ha presentado a inglés?

In [53]:
alumnosNoIngles = notas.keys().distinct().subtract(notasIng.keys())
alumnosNoIngles.collect()

['Pedro']

9. ¿A cuántas asignaturas se ha presentado cada alumno?

In [66]:
# nos quedamos con los nombres de cada alumno en cada asignatura y quitamos los repetidos. Luego los unimos
notasSinRep = notasMat.keys().distinct().union(notasFis.keys().distinct()).union(notasIng.keys().distinct())
notasSinRep.collect()

['Angel',
 'Ramon',
 'Jorge',
 'Susana',
 'Pedro',
 'Carlos',
 'Andres',
 'Leonardo',
 'Jose Juan',
 'Nicolas',
 'Maria',
 'Anabel',
 'Rocio',
 'Triana',
 'Fernando',
 'Oscar',
 'Isabel',
 'Alejandro',
 'Rosa',
 'Angel',
 'Ramon',
 'Jorge',
 'Susana',
 'Pedro',
 'Carlos',
 'Andres',
 'Leonardo',
 'Jose Juan',
 'Nicolas',
 'Maria',
 'Anabel',
 'Rocio',
 'Triana',
 'Fernando',
 'Oscar',
 'Isabel',
 'Alejandro',
 'Rosa',
 'Angel',
 'Ramon',
 'Jorge',
 'Susana',
 'Carlos',
 'Andres',
 'Leonardo',
 'Jose Juan',
 'Nicolas',
 'Maria',
 'Anabel',
 'Rocio',
 'Triana',
 'Fernando',
 'Oscar',
 'Isabel',
 'Alejandro',
 'Rosa']

In [65]:
alumnos = notasSinRep.map(lambda x: (x, 1)).reduceByKey(lambda a,b: a+b)
alumnos.collect()

[('Angel', 3),
 ('Carlos', 3),
 ('Anabel', 3),
 ('Jorge', 3),
 ('Susana', 3),
 ('Andres', 3),
 ('Jose Juan', 3),
 ('Rocio', 3),
 ('Fernando', 3),
 ('Oscar', 3),
 ('Isabel', 3),
 ('Alejandro', 3),
 ('Ramon', 3),
 ('Pedro', 2),
 ('Leonardo', 3),
 ('Nicolas', 3),
 ('Maria', 3),
 ('Triana', 3),
 ('Rosa', 3)]

10. Obten un RDD con cada alumno con sus notas

In [71]:
notasAlumnos = notas.groupByKey()
notasAlumnos = notasAlumnos.map(lambda x: (x[0], list(x[1])))
notasAlumnos.collect()

[('Angel', ['6', '9', '4']),
 ('Carlos', ['4', '4', '8']),
 ('Anabel', ['8', '2', '7']),
 ('Jorge', ['10', '5', '5']),
 ('Susana', ['9', '9', '2']),
 ('Andres', ['4', '4', '6']),
 ('Jose Juan', ['5', '3', '3']),
 ('Rocio', ['6', '5', '7', '4']),
 ('Fernando', ['5', '9', '7']),
 ('Oscar', ['7', '5', '3']),
 ('Isabel', ['8', '8', '7']),
 ('Alejandro', ['5', '3', '7']),
 ('Ramon', ['4.5', '7', '8']),
 ('Pedro', ['5', '2']),
 ('Leonardo', ['1', '6', '4']),
 ('Nicolas', ['2', '7', '5']),
 ('Maria', ['2', '3', '6']),
 ('Triana', ['3', '3', '4']),
 ('Rosa', ['6', '8', '9'])]